<table class="table table-bordered">
    <tr>
        <th style="width:250px"><img src='https://www.np.edu.sg/images/default-source/default-album/img-logo.png?sfvrsn=764583a6_0' style="width: 100%; height: 125px; "></th>
        <th style="text-align:center;"><h1>Distributed Data Pipelines</h1><h2>Assignment 1 </h2><h3>Diploma in Data Science</h3></th>
    </tr>
</table>

Learning Objectives:
- Design PySpark Based Machine Learning
- Execute PySpark Syntax Correctly
- Evaluate and Select Final Model based on Metrics

You will be **graded on the use of PySpark**, so usage of **Pandas itself should be avoided as much as possible**, especially if a particular native method or function is already available in PySpark. **Penalties will be imposed in such cases.**

In [1]:
# import the packages
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('6.1').getOrCreate()

In [2]:
# wider page width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

C:\Users\user\AppData\Local\Temp\ipykernel_25944\1745900340.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Table of Contents
- Step 1: [Problem Statement Formulation](#s1)
    -  [Value Based Problem Statement](#ps)
- Step 2: [Exploratory Data Analysis and Data Cleansing](#s2)
- Step 3: [Data Wrangling and Transformation](#s3)
- Step 4: [Machine Learning Modelling](#s4)
- Step 5: [Model Evaluation and Selection](#s5)
- Step 6: [Report](#rpt)
- Presentation Video: [Video Presentation](#vid)

## Step 1: Problem Statement Formulation <a name = "s1"></a>

In [3]:
# load and explore data
df_pyspark=spark.read.csv('./data/sg_flat_prices_mod.csv',
                          header=True, inferSchema=True) # inferSchema auto detects data type

In [4]:
# View Dataset Schema
df_pyspark.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- town: string (nullable = true)
 |-- flat_type: string (nullable = true)
 |-- block: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- storey_range: string (nullable = true)
 |-- floor_area_sqm: double (nullable = true)
 |-- flat_model: string (nullable = true)
 |-- lease_commence_date: integer (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)



In [5]:
# View top 10 rows in dataset
df_pyspark.show(10)

+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+
|year|month|      town|flat_type|block|      street_name|storey_range|floor_area_sqm|    flat_model|lease_commence_date|remaining_lease|resale_price|
+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|    10 TO 12|          44.0|      Improved|               1979|            736|    232000.0|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|    01 TO 03|          67.0|New Generation|               1978|            727|    250000.0|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|    01 TO 03|          67.0|New Generation|               1980|            749|    262000.0|
|2017|    1|ANG MO KIO|   3 ROOM|  465|ANG MO KIO AVE 10|    04 TO 06|          68.0|New Generation|

In [6]:
# Dimension of Dataframe
row = df_pyspark.count()
col = len(df_pyspark.columns)
 
# display
print(f'Number of rows: {row}')
print(f'Number of columns: {col}')
print(f'Shape of the Dataframe: {(row,col)}')


Number of rows: 64247
Number of columns: 12
Shape of the Dataframe: (64247, 12)


In [7]:
# Summary of first 6 columns
df_pyspark.describe(['year','month','town','flat_type','block','street_name']).show()

+-------+------------------+------------------+----------+----------------+------------------+------------+
|summary|              year|             month|      town|       flat_type|             block| street_name|
+-------+------------------+------------------+----------+----------------+------------------+------------+
|  count|             64247|             64247|     64247|           64247|             64247|       64247|
|   mean|2018.0262424704656| 6.779133656046197|      null|            null| 349.4635640052788|        null|
| stddev|0.8146939469668695|3.2635673352950514|      null|            null|254.80560486394563|        null|
|    min|              2017|                 1|ANG MO KIO|          1 ROOM|                 1|ADMIRALTY DR|
|    max|              2019|                12|    YISHUN|MULTI-GENERATION|                9B|     ZION RD|
+-------+------------------+------------------+----------+----------------+------------------+------------+



In [8]:
# Summary of next 6 columns
df_pyspark.describe(['storey_range','floor_area_sqm','flat_model','lease_commence_date','remaining_lease','resale_price']).show()

+-------+------------+-----------------+-------------+-------------------+------------------+------------------+
|summary|storey_range|   floor_area_sqm|   flat_model|lease_commence_date|   remaining_lease|      resale_price|
+-------+------------+-----------------+-------------+-------------------+------------------+------------------+
|  count|       64247|            64197|        64247|              64247|             64247|             64247|
|   mean|        null|97.77009984890256|         null| 1993.6012420813422| 894.6413840334957|438943.70469516085|
| stddev|        null|24.26994610142912|         null| 12.465629502278013|149.62669792791093|153760.65294972394|
|    min|    01 TO 03|             31.0|Adjoined flat|               1966|               553|          150000.0|
|    max|    49 TO 51|            249.0|      Type S2|               2016|              1160|         1205000.0|
+-------+------------+-----------------+-------------+-------------------+------------------+---

In [9]:
# Show unique values for categorical features (limit to 30) (storey_range, town, flat_model, flat_type, street_name)
# storey_range
df_pyspark.select('storey_range').distinct().collect()

[Row(storey_range='43 TO 45'),
 Row(storey_range='37 TO 39'),
 Row(storey_range='10 TO 12'),
 Row(storey_range='04 TO 06'),
 Row(storey_range='40 TO 42'),
 Row(storey_range='16 TO 18'),
 Row(storey_range='19 TO 21'),
 Row(storey_range='34 TO 36'),
 Row(storey_range='31 TO 33'),
 Row(storey_range='13 TO 15'),
 Row(storey_range='46 TO 48'),
 Row(storey_range='22 TO 24'),
 Row(storey_range='25 TO 27'),
 Row(storey_range='07 TO 09'),
 Row(storey_range='01 TO 03'),
 Row(storey_range='28 TO 30'),
 Row(storey_range='49 TO 51')]

In [10]:
# town
df_pyspark.select('town').distinct().collect()

[Row(town='QUEENSTOWN'),
 Row(town='BEDOK'),
 Row(town='CLEMENTI'),
 Row(town='SERANGOON'),
 Row(town='BUKIT PANJANG'),
 Row(town='BUKIT TIMAH'),
 Row(town='YISHUN'),
 Row(town='GEYLANG'),
 Row(town='WOODLANDS'),
 Row(town='BUKIT MERAH'),
 Row(town='TOA PAYOH'),
 Row(town='BISHAN'),
 Row(town='PUNGGOL'),
 Row(town='HOUGANG'),
 Row(town='ANG MO KIO'),
 Row(town='PASIR RIS'),
 Row(town='SENGKANG'),
 Row(town='KALLANG/WHAMPOA'),
 Row(town='BUKIT BATOK'),
 Row(town='TAMPINES'),
 Row(town='JURONG WEST'),
 Row(town='JURONG EAST'),
 Row(town='MARINE PARADE'),
 Row(town='CENTRAL AREA'),
 Row(town='SEMBAWANG'),
 Row(town='CHOA CHU KANG')]

In [11]:
# flat model
df_pyspark.select('flat_model').distinct().collect()

[Row(flat_model='Apartment'),
 Row(flat_model='Premium Maisonette'),
 Row(flat_model='Improved'),
 Row(flat_model='Type S2'),
 Row(flat_model='New Generation'),
 Row(flat_model='Improved-Maisonette'),
 Row(flat_model='Model A-Maisonette'),
 Row(flat_model='Maisonette'),
 Row(flat_model='Multi Generation'),
 Row(flat_model='Model A'),
 Row(flat_model='DBSS'),
 Row(flat_model='Simplified'),
 Row(flat_model='Terrace'),
 Row(flat_model='Adjoined flat'),
 Row(flat_model='Type S1'),
 Row(flat_model='Standard'),
 Row(flat_model='Premium Apartment'),
 Row(flat_model='Model A2'),
 Row(flat_model='Premium Apartment Loft')]

In [12]:
# flat type
df_pyspark.select('flat_type').distinct().collect()

[Row(flat_type='3 ROOM'),
 Row(flat_type='1 ROOM'),
 Row(flat_type='4 ROOM'),
 Row(flat_type='2 ROOM'),
 Row(flat_type='EXECUTIVE'),
 Row(flat_type='5 ROOM'),
 Row(flat_type='MULTI-GENERATION')]

In [13]:
# street name
df_pyspark.select('street_name').distinct().collect()

[Row(street_name='BEDOK STH AVE 3'),
 Row(street_name='JURONG EAST ST 32'),
 Row(street_name='NTH BRIDGE RD'),
 Row(street_name='BEDOK NTH ST 2'),
 Row(street_name='CRAWFORD LANE'),
 Row(street_name='ANG MO KIO ST 21'),
 Row(street_name='SEMBAWANG CRES'),
 Row(street_name='RIVERVALE DR'),
 Row(street_name='WOODLANDS DR 60'),
 Row(street_name='LIM LIAK ST'),
 Row(street_name='BEDOK NTH AVE 3'),
 Row(street_name='CHOA CHU KANG NTH 6'),
 Row(street_name='LOR 8 TOA PAYOH'),
 Row(street_name='JOO SENG RD'),
 Row(street_name='TAO CHING RD'),
 Row(street_name='CHOA CHU KANG ST 53'),
 Row(street_name='PUNGGOL PL'),
 Row(street_name='JLN DAMAI'),
 Row(street_name='MCNAIR RD'),
 Row(street_name='EVERTON PK'),
 Row(street_name='BT BATOK ST 25'),
 Row(street_name='JLN BT HO SWEE'),
 Row(street_name='KALLANG BAHRU'),
 Row(street_name='YISHUN ST 20'),
 Row(street_name='BEDOK NTH ST 4'),
 Row(street_name='SERANGOON NTH AVE 3'),
 Row(street_name='ANG MO KIO AVE 8'),
 Row(street_name='QUEEN ST'),
 Row(

### Value Based Problem Statement <a name = "ps"></a>

Value based problem statement:

Construct a model that predicts the resale prices of any given HDB resale transaction based on its characteristics.\
*characteristics can refer to (flat model, storey range, flat type)

## Step 2: Exploratory Data Analysis and Data Cleansing <a name = "s2"></a>

### Data Exploration & Analysis With PySpark

In [14]:
# Target variable is resale_price
# Explore the relationship between the features in the dataset that are relevant to the target.
# Take note of any trends or anomalies

### Explore Resale Price against Remaining Lease

In [15]:
# Resale Price and Remaining Lease (in months) top20
df_pyspark.select(["remaining_lease", "resale_price"]).groupBy('remaining_lease').avg().sort("remaining_lease", ascending=False).drop("avg(remaining_lease)").show()
# order by descending

+---------------+------------------+
|remaining_lease| avg(resale_price)|
+---------------+------------------+
|           1160|          695000.0|
|           1159|          462500.0|
|           1158|          861000.0|
|           1157|          839000.0|
|           1156| 957666.6666666666|
|           1155|          799000.0|
|           1154|          750808.0|
|           1153|          615777.6|
|           1152|          488000.0|
|           1151| 440590.6666666667|
|           1150|         476733.28|
|           1149| 415028.5714285714|
|           1148| 419412.0707070707|
|           1147|428649.15862068965|
|           1146|444071.22424242424|
|           1145|437010.03831417626|
|           1144|450892.23247232474|
|           1143|464206.31470588234|
|           1142|          443500.0|
|           1141| 449181.2867830424|
+---------------+------------------+
only showing top 20 rows



In [16]:
# Resale Price and Remaining Lease (in months) bottom20
df_pyspark.select(["remaining_lease", "resale_price"]).groupBy('remaining_lease').avg().sort("remaining_lease", ascending=True).drop("avg(remaining_lease)").show()
# order by asc

+---------------+------------------+
|remaining_lease| avg(resale_price)|
+---------------+------------------+
|            553|          258700.0|
|            554|245166.66666666666|
|            555|          256000.0|
|            556|          246000.0|
|            557|          265000.0|
|            558|231111.11111111112|
|            559|230857.14285714287|
|            560|236769.23076923078|
|            561|250857.14285714287|
|            562|221222.22222222222|
|            563|          269564.0|
|            564|258333.33333333334|
|            565|          256000.0|
|            566|230916.66666666666|
|            567|245333.33333333334|
|            568|245914.46153846153|
|            569|          246649.9|
|            570|239666.66666666666|
|            571| 242307.6923076923|
|            572| 247692.3076923077|
+---------------+------------------+
only showing top 20 rows



In [17]:
# Conclusion: It can be observed that a higher remaining lease tends to mean a higher resale price based on the comparison between the two tables above.

### Explore Resale Price against Floor Area

In [18]:
# Resale Price and Floor Area
df_pyspark.select(["floor_area_sqm", "resale_price"]).sort(["floor_area_sqm", "resale_price"], ascending=False).show(20)
# order by descending

+--------------+------------+
|floor_area_sqm|resale_price|
+--------------+------------+
|         249.0|   1053888.0|
|         237.0|   1185000.0|
|         215.0|    900000.0|
|         215.0|    888000.0|
|         215.0|    830000.0|
|         192.0|    850000.0|
|         192.0|    800000.0|
|         192.0|    800000.0|
|         192.0|    780000.0|
|         192.0|    778000.0|
|         192.0|    760000.0|
|         192.0|    760000.0|
|         192.0|    755000.0|
|         192.0|    750000.0|
|         192.0|    738000.0|
|         192.0|    733000.0|
|         192.0|    729000.0|
|         192.0|    700000.0|
|         190.0|    750000.0|
|         189.0|    840000.0|
+--------------+------------+
only showing top 20 rows



In [19]:
df_pyspark.select(["floor_area_sqm", "resale_price"]).sort(["floor_area_sqm", "resale_price"], ascending=True).show(20)
# order by descending

+--------------+------------+
|floor_area_sqm|resale_price|
+--------------+------------+
|          null|    275000.0|
|          null|    275000.0|
|          null|    300000.0|
|          null|    315000.0|
|          null|    320000.0|
|          null|    325000.0|
|          null|    328000.0|
|          null|    330000.0|
|          null|    330000.0|
|          null|    340000.0|
|          null|    350000.0|
|          null|    352000.0|
|          null|    355000.0|
|          null|    360000.0|
|          null|    360000.0|
|          null|    363000.0|
|          null|    365000.0|
|          null|    380000.0|
|          null|    383000.0|
|          null|    383500.0|
+--------------+------------+
only showing top 20 rows



In [20]:
# Conclusion: By comparing the 2 tables, we are able to confirm that the resale price of the flat is proportional to the floor area of the flat. 
# (more area = more expensive)

### Explore Resale Price by Town

In [21]:
# Resale Price by Town sorted in desc
df_pyspark.select(["town", "resale_price"]).groupBy('town').avg().sort("avg(resale_price)", ascending=False).show(30)

+---------------+------------------+
|           town| avg(resale_price)|
+---------------+------------------+
|    BUKIT TIMAH| 714816.9735449735|
|         BISHAN|   644789.63170347|
|   CENTRAL AREA| 623428.0220183487|
|    BUKIT MERAH| 564024.8797051979|
|     QUEENSTOWN| 554835.8535597189|
|  MARINE PARADE| 518115.9173126615|
|KALLANG/WHAMPOA|496043.73121085594|
|      TOA PAYOH| 494166.7532051282|
|      PASIR RIS| 492123.0871459695|
|      SERANGOON| 490769.0934438583|
|       TAMPINES|474205.32034313725|
|       CLEMENTI| 469028.6115466857|
|        PUNGGOL| 453269.6137253925|
|       SENGKANG|433994.11826156947|
|        GEYLANG|430605.67181467183|
|        HOUGANG|429212.74610328645|
|  BUKIT PANJANG|428196.38632550335|
|    JURONG EAST| 416185.7745504841|
|     ANG MO KIO| 411547.1964346932|
|          BEDOK| 410944.0495771362|
|    JURONG WEST| 387879.4531122346|
|  CHOA CHU KANG|384960.08120300755|
|      SEMBAWANG| 378804.1880729761|
|    BUKIT BATOK| 377715.2757037944|
|

In [22]:
# Conclusion: It can be observed that the top 3 towns with the highest resale value is Bukit Timah, Bishan, and the Central Area. 
# It may indicate that these are high value locations in terms of housing.

### Explore Resale Price by Flat Type

In [23]:
# Resale Price by Flat Type in desc
df_pyspark.select(["flat_type", "resale_price"]).groupBy('flat_type').avg().sort("avg(resale_price)", ascending=False).show(30)

+----------------+------------------+
|       flat_type| avg(resale_price)|
+----------------+------------------+
|MULTI-GENERATION| 806804.6060606061|
|       EXECUTIVE| 625390.6592435675|
|          5 ROOM| 528812.8562151295|
|          4 ROOM| 432760.1087740674|
|          3 ROOM| 307712.2837821541|
|          2 ROOM|233600.68988030468|
|          1 ROOM| 186181.6551724138|
+----------------+------------------+



In [24]:
# Conclusion: Comparing the flat types, it is clear that the most expensive flat type is a Multi-Generation Flat, while the least expensive would be a 1 Room Flat.

### Explore Resale Price by Storey Range

In [25]:
# Resale Price by Storey Range in desc
df_pyspark.select(["storey_range", "resale_price"]).groupBy('storey_range').avg().sort("avg(resale_price)", ascending=False).show(30)

+------------+------------------+
|storey_range| avg(resale_price)|
+------------+------------------+
|    43 TO 45|1037833.3333333334|
|    49 TO 51|1022814.6666666666|
|    46 TO 48|1018845.4545454546|
|    40 TO 42|       894045.9375|
|    37 TO 39| 845602.7674418605|
|    34 TO 36| 802757.8962962963|
|    31 TO 33| 800630.9291338583|
|    28 TO 30| 751391.7605177993|
|    25 TO 27| 666919.1401295896|
|    22 TO 24| 610122.5498092031|
|    19 TO 21| 591394.6781223805|
|    16 TO 18|514570.93940520444|
|    13 TO 15|472987.87508488825|
|    10 TO 12|438086.75250713184|
|    07 TO 09|423477.66865580005|
|    04 TO 06| 412115.9694568433|
|    01 TO 03| 394274.2836198463|
+------------+------------------+



In [26]:
# Conclusion: From the table, it can be observed that there is a trend where the flat is priced higher if it is located on a higher storey range.

### Explore Resale Price by Year

In [27]:
# Resale Price by Year
df_pyspark.select(["year", "resale_price"]).groupBy('year').avg().sort("avg(resale_price)", ascending=False).drop("avg(year)").show(30)

+----+------------------+
|year| avg(resale_price)|
+----+------------------+
|2017|443849.84801951225|
|2018|441282.06370344607|
|2019| 432137.9129018299|
+----+------------------+



In [28]:
# Conclusion: The table above seems to indicate that the average resale price of flats has decreased slightly from 2017 to 2019. 

### Determine relationship between block and resale price

In [29]:
# Relationship between block and resale price (top 10)
df_pyspark.select(["block", "resale_price"]).sort("resale_price", ascending=False).show(10)

+-----+------------+
|block|resale_price|
+-----+------------+
|    8|   1205000.0|
|   1C|   1200000.0|
|   9A|   1200000.0|
|   1C|   1188000.0|
|    9|   1185000.0|
|   41|   1185000.0|
| 273B|   1180000.0|
|  18C|   1170000.0|
|   1D|   1168000.0|
| 139A|   1160888.0|
+-----+------------+
only showing top 10 rows



In [30]:
# bottom 10
df_pyspark.select(["block", "resale_price"]).sort("resale_price", ascending=False).show(10)

+-----+------------+
|block|resale_price|
+-----+------------+
|    8|   1205000.0|
|   1C|   1200000.0|
|   9A|   1200000.0|
|   1C|   1188000.0|
|    9|   1185000.0|
|   41|   1185000.0|
| 273B|   1180000.0|
|  18C|   1170000.0|
|   1D|   1168000.0|
| 139A|   1160888.0|
+-----+------------+
only showing top 10 rows



In [31]:
# Conclusion: There does not seem to be any relationship between the block number and the resale price of a flat. 
# Therefore, we can safely drop this feature later as it does not seem to be a helpful predictor.

### Distribution of Numerical Features

In [32]:
# Distribution of Numerical Features
# Relevant features: (floor_area_sqm, remaining_lease, resale_price)
# date features and irrelevant features such as block & lease commencement will not give any insight
df_pyspark.describe(['floor_area_sqm', 'remaining_lease', 'resale_price']).show()

+-------+-----------------+------------------+------------------+
|summary|   floor_area_sqm|   remaining_lease|      resale_price|
+-------+-----------------+------------------+------------------+
|  count|            64197|             64247|             64247|
|   mean|97.77009984890256| 894.6413840334957|438943.70469516085|
| stddev|24.26994610142912|149.62669792791093|153760.65294972394|
|    min|             31.0|               553|          150000.0|
|    max|            249.0|              1160|         1205000.0|
+-------+-----------------+------------------+------------------+



#### Checking NaN

In [33]:
# Check Dataframe for NA values
from pyspark.sql.functions import col,isnan, when, count
df_pyspark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_pyspark.columns]).show()

+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+
|year|month|town|flat_type|block|street_name|storey_range|floor_area_sqm|flat_model|lease_commence_date|remaining_lease|resale_price|
+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+
|   0|    0|   0|        0|    0|          0|           0|            50|         0|                  0|              0|           0|
+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+



#### Handling Missing Values

In [34]:
# consider NaN Treatment
# Previously identified 50 rows with NaN in floor_area_sqm
# Considering we have 64247 rows, it is more preferable to drop the rows (have sufficient rows)
# This is to prevent us from introducing falsified data into the dataframe (using imputation)
# which can affect the accuracy and bias of our final model.
df_pyspark_nan = df_pyspark.na.drop(how="any")
df_pyspark_nan.show()

+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+
|year|month|      town|flat_type|block|      street_name|storey_range|floor_area_sqm|    flat_model|lease_commence_date|remaining_lease|resale_price|
+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|    10 TO 12|          44.0|      Improved|               1979|            736|    232000.0|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|    01 TO 03|          67.0|New Generation|               1978|            727|    250000.0|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|    01 TO 03|          67.0|New Generation|               1980|            749|    262000.0|
|2017|    1|ANG MO KIO|   3 ROOM|  465|ANG MO KIO AVE 10|    04 TO 06|          68.0|New Generation|

In [35]:
# Dimension of new Dataframe (no NaN)
row = df_pyspark_nan.count()
col = len(df_pyspark_nan.columns)
 
# display
print(f'Number of rows: {row}')
print(f'Number of columns: {col}')
print(f'Shape of the Dataframe: {(row,col)}')

Number of rows: 64197
Number of columns: 12
Shape of the Dataframe: (64197, 12)


### Drop Irrelevant Columns

In [36]:
# Remove irrelevant columns (only block, street_name, month, and lease_commence_date for now)
# will not be useful for prediction based on exploratory analysis.
df_pyspark_new = df_pyspark_nan.drop('block','lease_commence_date','street_name','month')
df_pyspark_new.show()

+----+----------+---------+------------+--------------+--------------+---------------+------------+
|year|      town|flat_type|storey_range|floor_area_sqm|    flat_model|remaining_lease|resale_price|
+----+----------+---------+------------+--------------+--------------+---------------+------------+
|2017|ANG MO KIO|   2 ROOM|    10 TO 12|          44.0|      Improved|            736|    232000.0|
|2017|ANG MO KIO|   3 ROOM|    01 TO 03|          67.0|New Generation|            727|    250000.0|
|2017|ANG MO KIO|   3 ROOM|    01 TO 03|          67.0|New Generation|            749|    262000.0|
|2017|ANG MO KIO|   3 ROOM|    04 TO 06|          68.0|New Generation|            745|    265000.0|
|2017|ANG MO KIO|   3 ROOM|    01 TO 03|          67.0|New Generation|            749|    265000.0|
|2017|ANG MO KIO|   3 ROOM|    01 TO 03|          68.0|New Generation|            756|    275000.0|
|2017|ANG MO KIO|   3 ROOM|    04 TO 06|          68.0|New Generation|            738|    280000.0|


## Step 3: Data Wrangling and Transformation <a name = "s3"></a>

### Train Test Split

In [37]:
df_pyspark_final = df_pyspark_new

In [38]:
# Train Test Split
(train, test) = df_pyspark_final.randomSplit([0.8, 0.2], seed=16)

In [39]:
# Show train type
print(type(train))

<class 'pyspark.sql.dataframe.DataFrame'>


In [40]:
# show train and test dimensions after split

In [41]:
# Display train
row = train.count()
col = len(train.columns)
print(f'Number of rows: {row}')
print(f'Number of columns: {col}')
print(f'Shape of the Dataframe: {(row,col)}')

Number of rows: 51337
Number of columns: 8
Shape of the Dataframe: (51337, 8)


In [42]:
# Display test
row = test.count()
col = len(test.columns)
print(f'Number of rows: {row}')
print(f'Number of columns: {col}')
print(f'Shape of the Dataframe: {(row,col)}')

Number of rows: 12860
Number of columns: 8
Shape of the Dataframe: (12860, 8)


In [43]:
# Display train rows (5)
train.show(5)

+----+----------+---------+------------+--------------+----------+---------------+------------+
|year|      town|flat_type|storey_range|floor_area_sqm|flat_model|remaining_lease|resale_price|
+----+----------+---------+------------+--------------+----------+---------------+------------+
|2017|ANG MO KIO|   2 ROOM|    01 TO 03|          44.0|  Improved|            743|    215000.0|
|2017|ANG MO KIO|   2 ROOM|    01 TO 03|          44.0|  Improved|            743|    220000.0|
|2017|ANG MO KIO|   2 ROOM|    01 TO 03|          45.0|  Improved|            815|    205000.0|
|2017|ANG MO KIO|   2 ROOM|    04 TO 06|          44.0|  Improved|            712|    233000.0|
|2017|ANG MO KIO|   2 ROOM|    04 TO 06|          44.0|  Improved|            713|    240000.0|
+----+----------+---------+------------+--------------+----------+---------------+------------+
only showing top 5 rows



In [44]:
# Display train rows (5)
test.show(5)

+----+----------+---------+------------+--------------+--------------+---------------+------------+
|year|      town|flat_type|storey_range|floor_area_sqm|    flat_model|remaining_lease|resale_price|
+----+----------+---------+------------+--------------+--------------+---------------+------------+
|2017|ANG MO KIO|   2 ROOM|    04 TO 06|          44.0|      Improved|            717|    210000.0|
|2017|ANG MO KIO|   2 ROOM|    07 TO 09|          44.0|      Improved|            713|    245000.0|
|2017|ANG MO KIO|   2 ROOM|    10 TO 12|          44.0|      Improved|            717|    250000.0|
|2017|ANG MO KIO|   3 ROOM|    01 TO 03|          67.0|New Generation|            695|    282000.0|
|2017|ANG MO KIO|   3 ROOM|    01 TO 03|          67.0|New Generation|            711|    325000.0|
+----+----------+---------+------------+--------------+--------------+---------------+------------+
only showing top 5 rows



### Categorical Transformation

In [45]:
# Import libraries (OHE & String Indexing)
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [46]:
# Define categorical features to be transformed
strings_used = ["town","flat_type","flat_model","storey_range"] # defines which column will be encoded add another one by adding the column name to the list

#### String Indexing

In [47]:
# Perform String Indexing for Categorical columns(town, flat type, flat model, storey range)
#stage_string = [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in strings_used]

In [48]:
# Fit to train
#train_string_indexed = stage_string.fit(train).transform(train)
#train_string_indexed.show()

In [49]:
# Fit to test
#test_string_indexed = stage_string.fit(test).transform(test)
#test_string_indexed.show()

#### One-Hot Encoding

In [50]:
# Perform One-Hot Encoding for Categorical columns(town, flat type, flat model, storey range)

In [51]:
#stage_one_hot = [OneHotEncoder(inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in strings_used]

In [52]:
# has to be string indexed first as OneHotEncoder only converts category indices and not the original categorical values
#train_ohe = stage_one_hot.fit(train_string_indexed).transform(train_string_indexed)
#train_ohe.show()

In [53]:
#test_ohe = stage_one_hot.fit(test_string_indexed).transform(test_string_indexed)
#test_ohe.show()

#### Encoding Pipeline (One-Hot Encoding)

In [54]:
# Pipeline (does both at once) (Full OHE)
strings_used = ["town","flat_type","flat_model","storey_range"] 

stage_string = [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in strings_used]
stage_one_hot = [OneHotEncoder(inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in strings_used]

ppl = Pipeline(stages= stage_string + stage_one_hot)
train_index_ohe = ppl.fit(train).transform(train)
test_index_ohe = ppl.fit(test).transform(test)
train_index_ohe.show(5)

+----+----------+---------+------------+--------------+----------+---------------+------------+-------------------+------------------------+-------------------------+---------------------------+--------------+-----------------+------------------+--------------------+
|year|      town|flat_type|storey_range|floor_area_sqm|flat_model|remaining_lease|resale_price|town_string_encoded|flat_type_string_encoded|flat_model_string_encoded|storey_range_string_encoded|  town_one_hot|flat_type_one_hot|flat_model_one_hot|storey_range_one_hot|
+----+----------+---------+------------+--------------+----------+---------------+------------+-------------------+------------------------+-------------------------+---------------------------+--------------+-----------------+------------------+--------------------+
|2017|ANG MO KIO|   2 ROOM|    01 TO 03|          44.0|  Improved|            743|    215000.0|                8.0|                     4.0|                      1.0|                        3.0|(2

#### Dropping features

In [55]:
# drop original columns from both (town, flat type, flat model, storey range)
# They have been encoded into numerical values suitable for machine learning modelling
train_index_ohe = train_index_ohe.drop('town','flat_type','flat_model','storey_range')
test_index_ohe = test_index_ohe.drop('town','flat_type','flat_model','storey_range')
train_index_ohe.show()

+----+--------------+---------------+------------+-------------------+------------------------+-------------------------+---------------------------+--------------+-----------------+------------------+--------------------+
|year|floor_area_sqm|remaining_lease|resale_price|town_string_encoded|flat_type_string_encoded|flat_model_string_encoded|storey_range_string_encoded|  town_one_hot|flat_type_one_hot|flat_model_one_hot|storey_range_one_hot|
+----+--------------+---------------+------------+-------------------+------------------------+-------------------------+---------------------------+--------------+-----------------+------------------+--------------------+
|2017|          44.0|            743|    215000.0|                8.0|                     4.0|                      1.0|                        3.0|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[3],[1.0])|
|2017|          44.0|            743|    220000.0|                8.0|                     4.0|             

In [56]:
# Keep OHE values only
train_index_ohe = train_index_ohe.select("year", "floor_area_sqm", "remaining_lease", "town_one_hot", "flat_type_one_hot", "flat_model_one_hot", "storey_range_one_hot", "resale_price")
test_index_ohe = test_index_ohe.select("year", "floor_area_sqm", "remaining_lease", "town_one_hot", "flat_type_one_hot", "flat_model_one_hot", "storey_range_one_hot", "resale_price")

### Numerical Transformation

#### Remaining Lease to Year (Final Keep Month)

In [57]:
# convert remaining lease into year instead of month (put in transformation)
import pyspark.sql.functions as f
from pyspark.sql.functions import col
train_index_t = train_index_ohe.withColumn("remaining_lease (year)", col("remaining_lease") / 12)
#train_index_t = train_index_ohe.withColumn("remaining_lease (year)", f.round('remaining_lease (year)',2)) do not round for more accurate values
#train_index_t = train_index_ohe.drop('remaining_lease')
train_index_t.show()

+----+--------------+---------------+--------------+-----------------+------------------+--------------------+------------+----------------------+
|year|floor_area_sqm|remaining_lease|  town_one_hot|flat_type_one_hot|flat_model_one_hot|storey_range_one_hot|resale_price|remaining_lease (year)|
+----+--------------+---------------+--------------+-----------------+------------------+--------------------+------------+----------------------+
|2017|          44.0|            743|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[3],[1.0])|    215000.0|    61.916666666666664|
|2017|          44.0|            743|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[3],[1.0])|    220000.0|    61.916666666666664|
|2017|          45.0|            815|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[3],[1.0])|    205000.0|     67.91666666666667|
|2017|          44.0|            712|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[0],[1.0])|    2330

In [58]:
test_index_t = test_index_ohe.withColumn("remaining_lease (year)", col("remaining_lease") / 12)
#test_index_t = test_index_ohe.drop('remaining_lease')
test_index_t.show()

+----+--------------+---------------+--------------+-----------------+------------------+--------------------+------------+----------------------+
|year|floor_area_sqm|remaining_lease|  town_one_hot|flat_type_one_hot|flat_model_one_hot|storey_range_one_hot|resale_price|remaining_lease (year)|
+----+--------------+---------------+--------------+-----------------+------------------+--------------------+------------+----------------------+
|2017|          44.0|            717|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[0],[1.0])|    210000.0|                 59.75|
|2017|          44.0|            713|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[1],[1.0])|    245000.0|    59.416666666666664|
|2017|          44.0|            717|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[2],[1.0])|    250000.0|                 59.75|
|2017|          67.0|            695|(25,[8],[1.0])|    (6,[2],[1.0])|    (18,[2],[1.0])|      (16,[3],[1.0])|    2820

#### Feature Assembler

In [59]:
# Consolidating Features into Xcols
# append target variable to the end
# Keep remaining_lease in month
train_index_t = train_index_t.select("year", "floor_area_sqm", "remaining_lease", "town_one_hot", "flat_type_one_hot", "flat_model_one_hot", "storey_range_one_hot", "resale_price")

test_index_t = test_index_t.select("year", "floor_area_sqm", "remaining_lease", "town_one_hot", "flat_type_one_hot", "flat_model_one_hot", "storey_range_one_hot", "resale_price")

from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=test_index_t.columns[:-1],outputCol="Xcols")

In [60]:
# Train
train_vectored = featureassembler.transform(train_index_t)

In [61]:
train_vectored.show(5)

+----+--------------+---------------+--------------+-----------------+------------------+--------------------+------------+--------------------+
|year|floor_area_sqm|remaining_lease|  town_one_hot|flat_type_one_hot|flat_model_one_hot|storey_range_one_hot|resale_price|               Xcols|
+----+--------------+---------------+--------------+-----------------+------------------+--------------------+------------+--------------------+
|2017|          44.0|            743|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[3],[1.0])|    215000.0|(68,[0,1,2,11,32,...|
|2017|          44.0|            743|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[3],[1.0])|    220000.0|(68,[0,1,2,11,32,...|
|2017|          45.0|            815|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[3],[1.0])|    205000.0|(68,[0,1,2,11,32,...|
|2017|          44.0|            712|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[0],[1.0])|    233000.0|(68,[0,

In [62]:
# Test
test_vectored = featureassembler.transform(test_index_t)

In [63]:
test_vectored.show(5)

+----+--------------+---------------+--------------+-----------------+------------------+--------------------+------------+--------------------+
|year|floor_area_sqm|remaining_lease|  town_one_hot|flat_type_one_hot|flat_model_one_hot|storey_range_one_hot|resale_price|               Xcols|
+----+--------------+---------------+--------------+-----------------+------------------+--------------------+------------+--------------------+
|2017|          44.0|            717|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[0],[1.0])|    210000.0|(68,[0,1,2,11,32,...|
|2017|          44.0|            713|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[1],[1.0])|    245000.0|(68,[0,1,2,11,32,...|
|2017|          44.0|            717|(25,[8],[1.0])|    (6,[4],[1.0])|    (18,[1],[1.0])|      (16,[2],[1.0])|    250000.0|(68,[0,1,2,11,32,...|
|2017|          67.0|            695|(25,[8],[1.0])|    (6,[2],[1.0])|    (18,[2],[1.0])|      (16,[3],[1.0])|    282000.0|(68,[0,

#### Feature Scaling on Xcols

In [64]:
from pyspark.ml.feature import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler

Scaler = StandardScaler(withMean=True, withStd=True, inputCol="Xcols", outputCol="Xcols_scaled")
yScaler = StandardScaler(withMean=True, withStd=True, inputCol="resale_price", outputCol="resale_price")
# Standard scaler uses Z-score to perform scaling (withMean=True, withStd=True,)
# MinMaxScaler rescales each feature individually to a common range [min, max] linearly
# Robust Scaler removes the median and scales the data according to the quantile range (Default IQR)
# MaxAbsScaler rescales each feature individually to range [-1, 1] by dividing through the largest maximum absolute value in each feature.

In [65]:
train_vectored = train_vectored.withColumn("resale_price",train_vectored.resale_price.cast('integer'))
test_vectored = test_vectored.withColumn("resale_price",test_vectored.resale_price.cast('integer'))

In [66]:
# Scale Train
train_scaled = Scaler.fit(train_vectored).transform(train_vectored)
ytrain_scaled = yScaler.fit(train_vectored).transform(train_vectored)
train_scaled.show()

IllegalArgumentException: requirement failed: Column resale_price must be of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually int.

In [ ]:
# Scale Test
test_scaled = Scaler.fit(test_vectored).transform(test_vectored)
ytest_scaled = yScaler.fit(test_vectored).transform(test_vectored)
test_scaled.show()

## Step 4: Machine Learning Modelling <a name = "s4"></a>

### Validate Data

In [ ]:
# use code to show number of rows and columns,
# as well as a sample of 10 rows before heading into Machine Learning Modelling

In [ ]:
# Select Data (train_data = final)
# train
train_data = train_scaled.select("Xcols_scaled","resale_price")
train_data.show(10)

In [ ]:
# test (test_data = final)
test_data = test_scaled.select("Xcols_scaled","resale_price")
test_data.show(10)

In [ ]:
# Show train final dimensions
row = train_data.count()
col = len(train_data.columns)
 
# display
print(f'Number of rows: {row}')
print(f'Number of columns: {col}')
print(f'Shape of the Dataframe: {(row,col)}')


In [ ]:
# Show test final dimensions
row = test_data.count()
col = len(test_data.columns)
 
# display
print(f'Number of rows: {row}')
print(f'Number of columns: {col}')
print(f'Shape of the Dataframe: {(row,col)}')

### Building a Linear Regression Model

In [ ]:
from pyspark.ml.regression import LinearRegression

regressor=LinearRegression(featuresCol="Xcols_scaled", labelCol='resale_price')
regressor=regressor.fit(train_data)

In [ ]:
type(regressor)

In [ ]:
# Coefficients
regressor.coefficients

In [ ]:
# Intercept
regressor.intercept

In [ ]:
# first row values of Xcols_sscaled
train_data.take(1)

## Step 5: Model Evaluation and Selection <a name = "s5"></a>

### Predicted Evaluation

In [ ]:
train_pred_results=regressor.evaluate(train_data)

In [ ]:
train_pred_results.predictions.show()

In [ ]:
test_pred_results=regressor.evaluate(test_data)

In [ ]:
test_pred_results.predictions.show()

### Model Metrics

In [ ]:
print(f'train MAE: {train_pred_results.meanAbsoluteError}')
print(f'train MSE: {train_pred_results.meanSquaredError}')
print(f'train R^2: {train_pred_results.r2}')

In [ ]:
print(f'test MAE: {test_pred_results.meanAbsoluteError}')
print(f'test MSE: {test_pred_results.meanSquaredError}')
print(f'test R^2: {test_pred_results.r2}')

In [ ]:
# Test on 5 rows

In [ ]:
pyspark_five_rows = test_data.limit(5)
pyspark_five_rows.show()

In [ ]:
regressor.evaluate(pyspark_five_rows).predictions.show()

## Step 6: Report <a name = "rpt"></a>

### Table of Contents
6.1 [Problem Statement Formulation](#psf)\
&emsp; 6.1.1 [Load, Explore and Understand Data](#leu)\
&emsp; 6.1.2 [Formulate Value Based Problem Statement](#fps)\
6.2 [Exploratory Data Analysis and Data Cleansing](#eda)\
&emsp; 6.2.1 [Interesting Trends and Anomalies](#ta)\
&emsp; 6.2.2 [Potential Errors & Handling Missing Values](#mis)\
6.3 [Data Wrangling and Transformation](#dw)\
&emsp; 6.3.1  [Dropping Irrelevant Features](#dw1)\
&emsp; 6.3.2  [Categorical Transformation](#dw2)\
&emsp; 6.3.3  [Numerical Transformation](#dw3)\
6.4 [Machine Learning Modelling](#ml)\
&emsp; 6.4.1  [Validating Data](#ml1)\
&emsp; 6.4.2  [Sample of 10 Rows Before Modelling](#ml2)\
&emsp; 6.4.3  [Building the Predictive Model](#ml3)\
6.5 [Model Evaluation and Selectionr](#eval)\
&emsp; 6.5.1  [Evaluation of Predictive Model](#eval1)\
&emsp; 6.5.2  [Compare Models and Final Model](#eval2)\
6.6 [Summary and Further Improvements](#sum)\
&emsp; 6.4.1  [Summary of Findings](#sum1)\
&emsp; 6.4.2  [Further Improvements](#sum2)

### 6.1 Problem Statement Formulation <a name = "psf"></a>

#### 6.1.1 Load, Explore and Understand Data<a name = "leu"></a>

This report aims to cover and explain my findings and the processes involved in building a machine learning model. 

Before modelling, we first have to formulate a value-based prediction problem statement to help us transform the data and build a model to evaluate the dataset. To form the problem statement, pyspark is employed to load and convert the csv file into a dataframe “df_pyspark” for exploration.

For exploration, functions such as .printSchema(), .show(), and .describe() was used to give an overview of the features present, the data types for each feature, and a brief statistical summary of the numerical features in the dataset. To display the dimensions of the dataset, a combination of .count and len() functions was used. Lastly, the unique values in the categorical features were identified using .distinct() and .collect().


#### 6.1.2 Formulate Value Based Problem Statement <a name = "fps"></a>

Based on the initial exploration of the dataset, the problem statement formed is “Construct a model that predicts the resale prices of any given HDB resale transaction based on its characteristics”, where characteristics can refer to flat model, storey range, flat type, etc. 

This model will be helpful to give potential buyers or sellers a rough estimate on the resale price of their HDB flat. 

### 6.2 Exploratory Data Analysis and Data Cleansing <a name = "eda"></a>

Exploratory Data Analysis (EDA) is conducted to better understand the trends, relationships, and potential errors within the dataset. Exploratory Data Analysis is done using pyspark tables using aggregate functions as well as functions such as .groupby() and .select().

Analysis of the features will be done in relation to the target, “resale_price”.


#### 6.2.1 Interesting Trends and Anomalies <a name = "ta"></a>

When exploring the features, a common trend observed is, flats are priced higher when remaining lease, floor area and storey range is higher. These are features that will be useful for model prediction. Another observation is that there is a downward trend in average resale price from 2017 to 2019.  

It can also be observed that the top 3 towns with the highest resale value is Bukit Timah, Bishan, and the Central Area. This may indicate that these are high value locations.

No connection was observed between the block number and resale price. Therefore, we can safely drop this feature later.

There is an anomaly in remaining lease as the values are in the hundreds despite the max lease being 99 years. After investigating, it was found that remaining lease was stored in months instead of years.

#### 6.2.2 Potential Errors & Handling Missing Values <a name = "mis"></a>

Missing values were also found in “floor_area_sqm”. There was a total of 50 missing rows in the dataset. Considering we have 64247 rows; it is preferable to drop the rows as we would still have sufficient data for prediction. Dropping is preferred as imputation may introduce falsified data which can affect the accuracy of and bias of the final model.

### 6.3 Data Wrangling and Transformation <a name = "dw"></a>

#### 6.3.1 Dropping Irrelevant Features <a name = "dw1"></a>

Before transformation, the data was further cleansed by removing the unnecessary features identified during that data exploration process. These features are,

-	“block”, which has little correlation with the target.
-	“lease_commence_date”, which has no purpose as “remaining_lease” is derived it.
-	“street_name” and “month” adds insignificant value in relation to the target.


#### 6.3.2 Categorical Transformation <a name = "dw2"></a>

For the categorical features, one-hot encoding was used to encode the categorical features into a distinct value readable by the model. To perform One-Hot encoding with pyspark, the features were encoded using string indexing with “StringIndexer”, transformed into a binary vector with One-hot Encoder. The Pipeline function was used to perform both steps on the categorical features, "town","flat_type","flat_model","storey_range". Finally, the “string_encoded” and original columns are dropped.

#### 6.3.3 Numerical Transformation <a name = "dw3"></a>

For numerical transformation, Vector Assembler and Feature Scaling was utilized. Vector Assembler was used to combine the columns, apart from “resale_price”, into a single vector column. This is done as Spark MLlib models only accept vectorized columns to maximize efficiency and scaling. 

Then scaling was applied onto the vectorized columns. Feature scaling is done to reduce the varying magnitude between features, to ensure that the weights of the model is not skewed. The following scalers were tested,

•	Standard Scaler uses Z-score to perform scaling.
•	Min-Max Scaler rescales each feature to a common range [min, max] linearly.
•	Robust Scaler removes the median and scales the data based on quantile range (Default IQR).
•	Max-Abs Scaler rescales each feature to range [-1, 1] by dividing through the largest maximum absolute value in each feature.


### 6.4 Machine Learning Modelling <a name = "ml"></a>

#### 6.4.1 Validating Data <a name = "ml1"></a>

Before inputting the train data into the model, the number of rows and columns of the train and test datasets were checked with “.count()” and “len()”. This is to validate the data and ensure that the predicted results are not trivial or unrealistic.

Train:
- Number of rows: 51337
- Number of columns: 2
- Shape of the Dataframe: (51337, 2)

Test
- Number of rows: 12860
- Number of columns: 2
- Shape of the Dataframe: (12860, 2)


#### 6.4.2 Sample of 10 Rows Before Modelling <a name = "ml2"></a>

These are the final features of the data that will be used for building the machine learning model.

- Predictor Features: "year", "floor_area_sqm", "remaining_lease", "town_one_hot", "flat_type_one_hot", "flat_model_one_hot", "storey_range_one_hot".

- Xcols_scaled: a vectorized column that consists of all the predictor features scaled using feature scaling. Used as the “featuresCol”.

- Target Variable: "resale_price". Used as the “labelCol”.


#### 6.4.3 Building the Predictive Model <a name = "ml3"></a>

The model used will be a Linear Regression model as it is the model best suited for the given scenario. This is because we are building a predictive model which predicts the numerical value for a target variable, “resale_price”, based on its predictor features, “Xcols_scaled”. The model is built with the “LinearRegression” function imported from Spark MLlib. The model is then trained and fitted with the train dataset.

### 6.5 Model Evaluation and Selection <a name = "eval"></a>

#### 6.5.1 Evaluation of Predictive Model <a name = "eval1"></a>

To evaluate the model, “regressor.evaluate()” and “.predictions.show()” was used on both the train and test data to obtain the model’s predicted values. This allows us to obtain metrics that will be useful for evaluating the model’s performance. Since we are building a linear regression model, we will be using Mean Squared Error, Mean Absolute Error, and R-Squared as the metrics to evaluate performance.

Mean Squared Error and Mean Absolute Error are based on the squared and absolute error between the observed and predicted resale price. Thus, the closer these values are to 0, the more accurate the model.

R-Squared determines the measure of variance between the dependent variable that is explained by the independent variables. It measures the goodness-of-fit for linear regression model from a range of 1 to 0, where 1 indicates a perfect model.


#### 6.5.2 Compare Models and Final Model <a name = "eval2"></a>

The models were tested with four different feature scalers as well as transforming “remaining_lease” from year to month to determine the best performing model based on the metrics.

Base Model (remaining_lease(rl) year,sscaler)

train MAE: 43368.04179190975

train MSE: 3182629561.931875

train R^2: 0.8654171556132857


test MAE: 54038.97493917037

test MSE: 5360853182.070648

test R^2: 0.7734896068378345

----------------------------

Standard Scaler Model (rl month,sscaler) (Slight improvement over rl year) (Best model accuracy)

train MAE: 43368.04169744031

train MSE: 3182629559.822617

train R^2: 0.8654171557024792

test MAE: 54038.97421693898

test MSE: 5360853041.811932

test R^2: 0.7734896127641405

----------------------------
Min Max Scaler Model (rl month,minmaxscaler) (worse than sscaler)

train MAE: 43363.725080087424

train MSE: 3182213456.2019987

train R^2: 0.8654347513439866

test MAE: 70326.91432223978

test MSE: 7938820136.060119

test R^2: 0.6645636041149474

------------------------------
Robust Scaler Model (rl month,robustscaler) (worse than sscaler)

train MAE: 83513.43817673266

train MSE: 13089819835.92039

train R^2: 0.4464749504310964

test MAE: 82451.0570283069

test MSE: 13463596906.583529

test R^2: 0.43112700066349974

------------------------------
Max Absolute Scaler (rl month,maxabsscaler) (worse than sscaler)

train MAE: 43363.72512272503

train MSE: 3182213456.2019687

train R^2: 0.865434751343988

test MAE: 76895.07507788035

test MSE: 9013418359.102718

test R^2: 0.6191589534509785


Comparing the results, we can see that the model using “remaining_lease” in month and scaled using Standard Scaler has the best overall performance among the models tested.

### 6.6 Summary and Further Improvements<a name = "sum"></a>

#### 6.6.1 Summary of Findings <a name = "sum1"></a>

Overall, the performance of the machine learning model built is satisfactory as it was able to achieve a test R-squared value of 0.7734 . Through data exploration and analysis, we were able to identify key trends as patterns that helped identify features that were useful for predicting the target variable based on the prediction problem statement. 

#### 6.6.2 Further Improvements <a name = "sum2"></a>

Based on the accuracy of the final model, there is definitely room for improvement as I believe the error present in the model can be further reduced to improve the performance.

Some possible improvements that can be made are increasing the complexity of the model by adding new features derived from the original features. This will allow our model to train on more data which may potentially increase the model’s performance. Enriching the dataset with data and features from other sources, such as “distance to nearest CC or MRT” or “last renovation date” can also help to further refine the model.

Lastly, model accuracy can be further optimized by testing other transformation methods such as outlier handling, normalizing, binning, and other numerical transformers such as log transformer, box-cox transformer, etc. To compare and find functions that best suit the data.


## "Unlisted" Youtube Link to Video Presentation <a name = "vid"></a>

In [ ]:
# insert your link in this cell, you are allowed to comment it out
# youtube link: 





